### Get Valid Movies

This file creates the metadata of only the movies that have cast information. Through this process, we remove movies that don't have appropriate x-ray data from our metadata. This file creates the file `movies_with_cast_manual.csv`

In [1]:
import pandas as pd

In [2]:
df_people = pd.read_csv("../data/6_character_metadata/all_people_with_duplicates.csv")

In [3]:
df_people.head()

,name_id,person,character,file
0,nm1799971,Marta Hazas,Rafaella,0_My_Fault
1,nm2338819,Eve Ryan,Betty,0_My_Fault
2,nm5913850,Nicole Wallace,Noah,0_My_Fault
3,nm1293644,Ivan MassaguÃ©,Padre Noah,0_My_Fault
4,nm1107138,Mariano Venancio,Martin,0_My_Fault


In [5]:
df_people['file'].nunique()

3570

In [6]:
import os

all = []
for dir in ["com", "in2010s", "before2010", "after2020"]:
    # all = all + os.listdir(f"../data/4_parsed_xrays/{dir}")
    df = pd.read_csv(f"../data/4_parsed_xrays/{dir}_sub_movies_with_xrays.csv")
    df['dir'] = dir
    all.append(df)

In [7]:
df_all_merged = pd.concat(all)

In [8]:
df_all_merged.shape

(3570, 12)

Some movies didn't have their title which is necessary for imdb matching in their metadata. Therefore, we use the metadata from `2_metadata`.

In [11]:
for dir in df_all_merged['dir'].unique():
    file_with_title = pd.read_csv(f"../data/2_metadata/{dir}/clean_meta_en_prime.csv")[['file', 'title']]
    print(dir)
    df_missing = df_all_merged.loc[(df_all_merged['title'].isnull()) & (df_all_merged['dir'] == dir)]
    
    filtered = file_with_title[file_with_title['file'].isin(df_missing['file'])]
    df_all_merged.loc[(df_all_merged['title'].isnull()) & (df_all_merged['dir'] == dir), 'title'] = filtered['title'].values

com
in2010s
before2010
after2020


In [12]:
df_all_merged['title'].isnull().sum()

0

In [13]:
SAVE_DIR = "../data/6_character_metadata"

In [16]:
df_all_merged[df_all_merged['file'].isin(df_people['file'])].to_csv(f"{SAVE_DIR}/movies_with_cast_manual.csv", index=False)